In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/MyDrive/Data/ProcessedData/World/TimeSeries_long_normalized.csv')
df = df.drop('Unnamed: 0',axis=1)
df.head()

,OMER,CGAU,BFC,US.MI,001979.SZ,HNI,AQB,KRC,URI,TRUE.BK,...,INVE,ITUB,KOS,9983.T,AKSO.OL,RPOWER.NS,STT,EDIT,0267.HK,CPF
0,0.092860,-0.155400,-0.222418,-0.221727,0.350624,-0.053565,-0.027542,-0.006067,-0.147703,0.132025,...,-0.187554,0.184951,0.052283,-0.229702,0.250994,0.617041,0.179613,0.075672,0.135217,0.054827
1,0.134074,-0.173922,-0.222418,-0.236610,0.285613,-0.062703,-0.020001,-0.014035,-0.138377,0.110814,...,-0.177317,0.218668,0.064642,-0.250132,0.233283,0.575858,0.197100,0.087706,0.085037,0.065152
2,0.096232,-0.171178,-0.227996,-0.231649,0.192479,-0.047854,-0.020329,-0.013712,-0.133865,0.110814,...,-0.178023,0.178665,0.025507,-0.255119,0.253549,0.579026,0.188658,0.105370,0.077657,0.074082
3,0.077499,-0.161574,-0.227996,-0.239693,0.173488,-0.065940,-0.021968,-0.037938,-0.157211,0.121420,...,-0.183318,0.165521,0.014178,-0.257613,0.264618,0.608593,0.145759,0.067944,0.082085,0.052595
4,0.081246,-0.167748,-0.225255,-0.231649,0.121260,-0.019868,-0.018690,-0.019311,-0.152638,0.121420,...,-0.183318,0.186094,0.028597,-0.270401,0.255763,0.665616,0.158422,0.072470,0.051092,0.067385


In [5]:
names = np.load('/content/drive/MyDrive/Data/ProcessedData/World/Names/3300_names_id.npy')

In [6]:
df = df.iloc[:, names]

In [7]:
import numpy as np
from scipy.stats import gaussian_kde

def CCD(X, Y):

    n = len(X)

    # Estimate marginal CDFs
    F_X = np.zeros(n)
    F_Y = np.zeros(n)
    for i in range(n):
        F_X[i] = (i+1) / n
        F_Y[i] = (i+1) / n

    # Estimate joint CDF
    xy = np.vstack([X,Y])
    kde = gaussian_kde(xy)
    cdf = np.zeros(n)
    for i in range(n):

        cdf[i] = kde.integrate_box([X[0], Y[0]], [X[i], Y[i]])/(i+1)

    # Calculate CCD
    ccd = 0
    for i in range(n):
        ccd += (cdf[i] - F_X[i]*F_Y[i])**2

    ccd = 1 - ccd/(2*n)

    return ccd

In [8]:
# Calculate CCD adjacency matrix
p = df.shape[1]
A = np.zeros((p,p))

In [ ]:
for k in range(0,p):
    print("K= ",k," out of ",p)
    i = k+1
    for j in tqdm(range(i+1,p)):
        A[(i-1),(j-1)] = CCD(df.iloc[:,i].to_numpy(), df.iloc[:,j].to_numpy())
        A[(j-1),(i-1)] = A[(i-1),(j-1)]

K=  0  out of  3294


 91%|█████████ | 2987/3292 [5:29:55<38:20,  7.54s/it]

In [ ]:
A

In [ ]:
A.mean()

In [ ]:
# Sparse thresholding ?
A[A < A.mean()] = 0


In [ ]:
# Extract K leading eigenvectors
K = 5
U, s, Vh = np.linalg.svd(A)
U_k = U[:,:K]


In [ ]:
# Get ratio matrix
T = np.log(p)
R = U_k[:,1:] / np.repeat(U_k[:,0].reshape(-1,1), K-1, axis=1)
R[abs(R) > T] = np.sign(R[abs(R) > T]) * T

In [ ]:
from sklearn.cluster import KMeans

# k-means clustering
kmeans = KMeans(n_clusters=K).fit(R)
labels = kmeans.labels_

In [ ]:
labels